# Aula 4: WebSockets em Python

Bem-vindo à quarta aula do curso de APIs em Python. Hoje abordaremos comunicação em tempo real usando WebSockets:


## 1. Sockets (Network Sockets)

### 1.1 O que é um socket?

* Um **socket** é uma abstração de software que representa um “ponto de extremidade” na comunicação entre processos (processes) — seja entre processos na mesma máquina ou entre máquinas diferentes.
* Na prática, um socket é identificado por um **endereço IP** + **porta TCP (ou UDP)**. Por exemplo, `192.168.0.10:5000` define um socket que escuta na porta 5000 do host cujo IP é 192.168.0.10.

### 1.2 Protocolos de transporte

Existem dois protocolos principais que “rodarão sobre” sockets:

1. **TCP (Transmission Control Protocol)**

   * Orientado a conexão (“connection-oriented”). Antes de trocar dados, cliente e servidor estabelecem uma **conexão confiável** (handshake em três etapas).
   * Garante entrega ordenada e sem duplicação de pacotes. Se algum pacote se perde, o TCP reenvia.
   * Indicado para aplicações que não podem tolerar perda (por exemplo, HTTP, FTP, SSH).

2. **UDP (User Datagram Protocol)**

   * Sem conexão (“connectionless”). O remetente envia datagramas a qualquer momento, sem handshake.
   * Não há garantia de entrega, ordem ou duplicação: pacotes podem chegar fora de ordem ou se perder.
   * Usado quando latência e baixa sobrecarga são mais importantes do que confiabilidade (por exemplo, streaming de vídeo/audio em tempo real, jogos online).

### 1.3 Funcionamento básico (TCP)

1. **Servidor** cria um socket TCP, faz bind a uma porta e chama `listen()` para aguardar conexões.
2. **Cliente** cria um socket TCP e chama `connect(endereço_do_servidor)`.
3. Quando o servidor “accepta” essa conexão, dois sockets (um no cliente, outro no servidor) são emparelhados.
4. Agora há um canal bidirecional confiável:

   * O cliente pode `send()` bytes e o servidor pode `recv()`, e vice-versa.
   * Essa conexão permanece aberta até que qualquer dos dois lados chame `close()`.

### 1.4 Vantagens e limitações

* **Vantagens**

  * Controle total sobre envio/recebimento de bytes.
  * Baixa sobrecarga (raw I/O); ideal para protocolos personalizados.

* **Limitações**

  * É necessário implementar lógica de protocolo por cima (por exemplo, convenção de quantos bytes enviamos para representar mensagem X).
  * Em aplicações web, o TCP puro não entende HTTP ou outros protocolos de alto nível, então precisamos de camadas adicionais para interpretar solicitações e respostas.

---

## 2. WebSocket

### 2.1 O que é WebSocket?

* **WebSocket** é um protocolo padronizado (RFC 6455) para permitir comunicação **full-duplex** (bidirecional) sobre uma única conexão TCP.
* Ao contrário do HTTP “padrão” (onde o cliente faz requisição e o servidor só responde e fecha a conexão), o WebSocket **mantém o canal aberto** depois de um handshake inicial via HTTP.
* Isso possibilita que tanto cliente quanto servidor enviem mensagens a qualquer momento, sem a sobrecarga de abrir/fechar conexões continuamente.

### 2.2 Fluxo de handshake

1. **Cliente HTTP abre a conexão** enviando cabeçalhos especiais:

   ```
   GET /chat HTTP/1.1
   Host: exemplo.com
   Upgrade: websocket
   Connection: Upgrade
   Sec-WebSocket-Key: <chave_base64>
   Sec-WebSocket-Version: 13
   ```
2. **Servidor WebSocket responde** validando a chave e confirmando o upgrade:

   ```
   HTTP/1.1 101 Switching Protocols
   Upgrade: websocket
   Connection: Upgrade
   Sec-WebSocket-Accept: <chave_retornada>
   ```
3. A partir desse ponto, **o protocolo muda** de HTTP para WebSocket. A conexão TCP continua aberta, mas os quadros que circulam não seguem mais cabeçalhos HTTP — eles têm seu próprio formato para marcar início, tamanho e fechamento de cada mensagem.

### 2.3 Comunicação Full-Duplex

* **Mensagem do Cliente → Servidor**

  * O navegador (ou qualquer cliente WebSocket) empacota o texto/binário na estrutura de quadro WebSocket e manda ao servidor pela mesma conexão TCP.
* **Mensagem do Servidor → Cliente**

  * O servidor pode “empurrar” dados para o cliente sem esperar requisição prévia. Basta usar o mesmo socket WebSocket para enviar um quadro de texto ou binário.
* **Persistência**

  * A conexão permanece aberta enquanto nenhuma das pontas a fechar explicitamente (ou ocorra timeout).
  * É muito mais eficiente para cenários em que há várias trocas rápidas de mensagens (chat, jogos, dashboards em tempo real).

### 2.4 Exemplos de uso

* **Chat em tempo real**

  * Em vez de cliente ficar “polling” o servidor via HTTP a cada segundo, o WebSocket entrega cada mensagem assim que ela é enviada pelo usuário.
* **Aplicações financeiras**

  * Atualizar cotações de ações em tempo real sem requerer nova requisição HTTP a cada segundo.
* **Jogos multiplayer**

  * Jogadores trocam informações de estado (movimento, ações) quase que instantaneamente.

### 2.5 Vantagens e limitações

* **Vantagens**

  * Baixa latência (sem overhead de abrir nova requisição HTTP).
  * Suporta comunicação bidirecional assíncrona, facilitando notificações “push”.

* **Limitações**

  * Alguns proxies/firewalls ainda podem não suportar ou filtrar WebSockets.
  * É necessário manter conexões abertas no servidor, exigindo mais gestão de recursos (threads, processos ou loops assíncronos).
  * Por ser “raw” (apenas quadros de texto/binário), você mesmo deve gerenciar serialização de mensagens (JSON, por exemplo) e tratamento de eventos.

---

## 3. Socket.IO

### 3.1 O que é Socket.IO?

* **Socket.IO** não é apenas um protocolo, mas sim **uma biblioteca** de JavaScript/Node.js (e implementações em diversas linguagens) que simplifica o uso de WebSockets e fallback para outros transportes quando WebSocket não estiver disponível.
* Ele adiciona camadas de funcionalidade sobre a conexão “bruta” de WebSocket, como:

  * **Reconexão automática** (quando o cliente perder conexão, o Socket.IO tenta reconectar sem o desenvolvedor precisar tratar manualmente).
  * **Namespaces** (contextos lógicos, como `/chat`, `/notificações`), permitindo separar canais de comunicação.
  * **Rooms** (salas), facilitando broadcast para grupos específicos de clientes.
  * **Eventos personalizáveis**, em vez de só “enviar mensagens de texto” ou “quadros binários”. Exemplo: `socket.emit('novo usuário', { nome: 'Alice' })`.

### 3.2 Transportes e fallback

* **Transporte principal**:

  1. Tenta usar WebSocket (se suportado pelo navegador e pela infraestrutura).
  2. Se falhar (por exemplo, algum firewall bloqueando), recorre a HTTP Long-Polling (o cliente faz requisição e servidor retém a resposta até ter novos dados, simulando “push”).
  3. Outros fallback podem incluir JSONP-Polling ou Flash sockets (mais raros hoje em dia).
* Esse mecanismo de fallback torna o Socket.IO **mais robusto** em ambientes instáveis de rede, garantindo que a aplicação espere sempre alguma forma de “conexão em tempo real” do cliente.

### 3.3 Principais diferenças em relação ao WebSocket “puro”

| Aspecto                 | WebSocket “puro”                               | Socket.IO                                                                     |
| ----------------------- | ---------------------------------------------- | ----------------------------------------------------------------------------- |
| **Transporte**          | Somente WebSocket (sem fallback)               | WebSocket + fallback (Long-Polling, etc.)                                     |
| **Eventos nomeados**    | Você próprio padroniza mensagens               | `socket.emit('evento', dados)` e `socket.on('evento', callback)`              |
| **Reconexão**           | Precisa programar manualmente                  | Reconexão automática embutida                                                 |
| **Namespaces**          | Não há por padrão                              | Sim: `/chat`, `/notificações`, etc.                                           |
| **Rooms/Salas**         | Precisa implementar lógica adicional           | Embutido (`join`, `leave`, broadcast em rooms\`)                              |
| **Handshake**           | Apenas o upgrade HTTP/WS                       | Primeiro passa por um “handshake” próprio, depois usa WebSocket se disponível |
| **Bibliotecas cliente** | API padrão do navegador (`new WebSocket(...)`) | Inclui client JavaScript com métodos extras (`io()`)                          |

### 3.4 Vantagens e desvantagens

* **Vantagens**

  * Recurso “pronto para produção” (fallback + reconexão).
  * API mais amigável para eventos nomeados e agrupamento em salas.
  * Simplifica gestão de múltiplos canais (“namespaces”) e broadcast seletivo.

* **Desvantagens**

  * **Sobrecarrega** um pouco mais em relação ao WebSocket puro, pois adiciona protocolo próprio acima do WebSocket (mais bytes de cabeçalho, eventos serialize/deserialização).
  * Você **depende** de servidores compatíveis com Socket.IO (não basta um simples servidor WebSocket).
  * Não é “padrão” do W3C; se você quiser interoperabilidade com clientes WebSocket “puros” nativos, precisará padronizar mensagens.

---

## 4. Resumo comparativo

1. **Network Socket (TCP/UDP)**

   * Nível mais baixo: controle de conexão, confiabilidade (TCP) ou não (UDP).
   * Você decide o protocolo (por exemplo, HTTP, FTP ou protocolo próprio).
2. **WebSocket**

   * Protocolo padrão que “prolonga” (upgrade) a conexão HTTP para um canal full-duplex.
   * Requer suporte tanto no cliente (navegador ou biblioteca) quanto no servidor.
   * Ideal para cenários em que o cliente e servidor precisam trocar mensagens frequentes e rápidas.
3. **Socket.IO**

   * Biblioteca de alto nível que “embrulha” WebSocket + fallback, fornece eventos nomeados, rooms e namespaces.
   * Facilita muito a vida do desenvolvedor ao lidar com reconexão, broadcast, tópicos, etc.
   * Exige que tanto cliente quanto servidor usem a mesma biblioteca (o protocolo do Socket.IO não é exatamente WebSocket puro).

---

### Quando usar cada um?

* **Somente Network Sockets (TCP/UDP)**

  * Quando precisar de controle total sobre bits e pacotes, escrevendo protocolo próprio.
  * Em aplicações desktop ou sistemas embarcados que não usam HTTP ou WebSocket.

* **WebSocket (puro)**

  * Quando você quer um canal bidirecional leve para aplicações web, sem depender de bibliotecas extras.
  * Bibliotecas populares: `websockets` (Python), `ws` (Node.js), suporte nativo em navegadores (`new WebSocket(url)`).

* **Socket.IO**

  * Quando você precisa de recursos mais avançados (reconexão automática, salas, namespaces) sem implementar tudo do zero.
  * Ideal para chat, dashboards em tempo real, notificações, jogos simples em tempo real, etc., onde a robustez e a simplicidade de uso importam mais que usar o protocolo WebSocket “estrito”.

---



## 2. Usando a biblioteca `websockets`

Instalação:

```bash
pip install websockets
```

### 2.1. Servidor WebSocket simples

In [ ]:
import asyncio
import websockets

async def echo(websocket):
    async for message in websocket:
        print(f"Recebido: {message}")
        await websocket.send(f"Echo: {message}")

async def main():
    # Aqui, serve() retorna um objeto de servidor dentro de um loop em execução
    async with websockets.serve(echo, 'localhost', 8765):
        print("Servidor WebSocket rodando em ws://localhost:8765")
        # Mantém o servidor ativo para sempre (ou até interrupção manual)
        await asyncio.Future()

if __name__ == "__main__":
    # Cria o event loop, executa “main()” dentro dele e fecha ao final
    asyncio.run(main())


**Explicação**

- `websockets.serve` cria servidor escutando na porta 8765.
- Handler `echo` recebe mensagens e envia de volta.
- `async for message`: mantém conexão lendo mensagens até desconexão.

### 2.2. Cliente WebSocket simples

In [ ]:
import asyncio
import websockets

async def hello():
    uri = "ws://localhost:8765"
    async with websockets.connect(uri) as websocket:
        await websocket.send("Olá servidor!")
        response = await websocket.recv()
        print(f"Resposta: {response}")

if __name__ == "__main__":
    asyncio.run(hello())

## 3. Integração com Flask usando Flask-SocketIO

Instalação:

```bash
pip install flask-socketio
pip install eventlet  # backend assíncrono para produção
```

In [ ]:
# app.py
from flask import Flask, render_template
from flask_socketio import SocketIO, send, emit, join_room, leave_room

app = Flask(__name__)
app.config['SECRET_KEY'] = 'chave_secreta'
socketio = SocketIO(app, async_mode='eventlet')

@app.route('/')
def index():
    return render_template('index.html')

@socketio.on('message')
def handle_message(msg):
    print(f"Mensagem recebida: {msg}")
    send(msg, broadcast=True)

@socketio.on('join')
def on_join(data):
    room = data['room']
    join_room(room)
    send(f"Entrou na sala {room}", room=room)

@socketio.on('leave')
def on_leave(data):
    room = data['room']
    leave_room(room)
    send(f"Saiu da sala {room}", room=room)

if __name__ == '__main__':
    socketio.run(app, host='0.0.0.0', port=5000)

## 4. Exemplo prático de chat em tempo real

Crie o arquivo `templates/index.html` com o cliente Socket.IO:

```html
<!DOCTYPE html>
<html>
<head>
  <title>Chat em Tempo Real</title>
</head>
<body>
  <ul id="messages"></ul>
  <input id="input" autocomplete="off"/><button id="send">Enviar</button>
  <script src="https://cdn.socket.io/4.5.0/socket.io.min.js"></script>
  <script>
    const socket = io();
    const input = document.getElementById('input');
    document.getElementById('send').onclick = () => {
      socket.send(input.value);
      input.value = '';
    };
    socket.on('message', (msg) => {
      const li = document.createElement('li');
      li.textContent = msg;
      document.getElementById('messages').append(li);
    });
  </script>
</body>
</html>
```

## 5. Gerenciamento de Salas e Broadcast

- Use `join_room(room)` e `leave_room(room)` para gerenciar salas.
- `send(..., room=room)` para enviar a um room específico.
- `broadcast=True` envia para todos conectados.

In [ ]:
# app.py
import asyncio
from flask import Flask, render_template, request
from flask_socketio import SocketIO, send, emit, join_room, leave_room

app = Flask(__name__)
app.config['SECRET_KEY'] = 'chave_secreta'
# Usa eventlet como backend assíncrono
socketio = SocketIO(app, async_mode='eventlet')

# Lista fixa (poderia vir de DB) de salas disponíveis
CHAT_ROOMS = ['sala1', 'sala2', 'sala3']

@app.route('/')
def index():
    """
    Renderiza o HTML principal. O cliente deve escolher um nome de usuário
    e então se conecta via Socket.IO, entra automaticamente no 'lobby'.
    """
    return render_template('index.html', rooms=CHAT_ROOMS)


@socketio.on('connect')
def handle_connect():
    """
    Quando o cliente se conecta, não sabemos ainda qual é o nome dele
    (será enviado depois pelo cliente). No entanto, podemos colocar
    o socket na sala 'lobby' imediatamente. A notificação de 'novo membro'
    no lobby acontecerá quando o cliente enviar o próprio nome via evento 'join_lobby'.
    """
    join_room('lobby')
    # Podemos opcionalmente emitir lista de salas atuais para esse cliente
    emit('rooms_list', {'rooms': CHAT_ROOMS}, room=request.sid)


@socketio.on('disconnect')
def handle_disconnect():
    """
    Quando o cliente desconecta, precisamos notificar as salas em que ele estava.
    Idealmente o cliente diria em qual sala estava; como simplificação, informaremos
    a todos na lobby que o usuário saiu (caso tenhamos guardado o nome dele na sessão).
    """
    username = request.args.get('username')  # Vem do query string, se foi passada
    # Esse emit no lobby é apenas ilustrativo; normalmente removeríamos usuário de rooms.
    if username:
        send(f"{username} desconectou.", room='lobby')


@socketio.on('join_lobby')
def handle_join_lobby(data):
    """
    Dados esperados: {'username': 'Alice'}
    Cliente ingressa no lobby (caso não esteja). Todos no lobby são notificados.
    """
    username = data.get('username', 'Anônimo')
    # Armazena o nome na sessão do socket (para referência futura)
    request.environ['username'] = username

    # Caso o usuário já estivesse em outra sala, deixamos explicitamente:
    for room in CHAT_ROOMS + ['lobby']:
        if room in socketio.server.manager.rooms.get('/', {}).get(request.sid, set()):
            leave_room(room)

    # Agora, ingresa no lobby
    join_room('lobby')
    send(f"{username} entrou no Lobby.", room='lobby')

    # Envia lista de salas ao cliente (por precaução)
    emit('rooms_list', {'rooms': CHAT_ROOMS}, room=request.sid)


@socketio.on('join_room')
def handle_join_room(data):
    """
    Dados esperados: {'username': 'Alice', 'room': 'sala1'}
    Cliente solicita entrar em “room”. Deve sair do lobby (ou de qualquer outra sala)
    antes de ingressar.
    """
    username = data.get('username', 'Anônimo')
    room = data.get('room')
    if room not in CHAT_ROOMS:
        emit('error_message', {'msg': f"A sala '{room}' não existe."}, room=request.sid)
        return

    # Sai de todas as salas conhecidas (incluindo lobby e outras salas)
    for r in CHAT_ROOMS + ['lobby']:
        if r in socketio.server.manager.rooms.get('/', {}).get(request.sid, set()):
            leave_room(r)

    # Agora ingresa na sala escolhida
    join_room(room)
    send(f"{username} entrou na sala {room}.", room=room)


@socketio.on('leave_room')
def handle_leave_room(data):
    username = data.get('username', 'Anônimo')
    room = data.get('room')
    if room == 'lobby':
        leave_room('lobby')
        send(f"{username} saiu do Lobby.", room='lobby')
        return
    if room not in CHAT_ROOMS:
        emit('error_message', {'msg': f"A sala '{room}' não existe."}, room=request.sid)
        return
    leave_room(room)
    send(f"{username} saiu da sala {room}.", room=room)
    join_room('lobby')
    send(f"{username} voltou ao Lobby.", room='lobby')


@socketio.on('send_message')
def handle_send_message(data):
    """
    Dados esperados: {'username': 'Alice', 'room': 'sala1', 'message': 'Oi pessoal!'}
    O cliente informa para qual sala mandar a mensagem. Se estiver em 'lobby',
    room='lobby'. Caso contrário, room='salaX'.
    """
    username = data.get('username', 'Anônimo')
    room = data.get('room', 'lobby')
    message = data.get('message', '')

    # Envia para todas as pessoas na mesma “room”
    send(f"{username}: {message}", room=room)


if __name__ == '__main__':
    # Nota: usar eventlet ou gevent para produção; aqui apenas ilustramos
    socketio.run(app, host='0.0.0.0', port=5000)


````javascript
<!-- templates/index.html -->
<!DOCTYPE html>
<html lang="pt-BR">
<head>
  <meta charset="UTF-8" />
  <title>Chat com Lobby e Múltiplas Salas</title>
  <style>
    body { font-family: Arial, sans-serif; margin: 20px; }
    #login, #chat-area { max-width: 500px; margin: auto; }
    #chat-area { display: none; }
    #rooms { margin-bottom: 10px; }
    #messages { list-style: none; padding: 0; height: 300px; overflow-y: scroll; border: 1px solid #ccc; }
    #messages li { padding: 5px 10px; }
    #input-container { display: flex; margin-top: 10px; }
    #input { flex: 1; padding: 5px; }
    #send { padding: 5px 10px; }
    .room-button { margin-right: 5px; }
    #current-room { font-weight: bold; margin-bottom: 10px; }
  </style>
  <script src="https://cdn.socket.io/4.5.0/socket.io.min.js"></script>
</head>
<body>
  <div id="login">
    <h2>Bem-vindo ao Chat</h2>
    <label>Escolha um nome de usuário:</label><br />
    <input type="text" id="username" placeholder="Digite seu nome" />
    <button id="btn-login">Entrar</button>
  </div>

  <div id="chat-area">
    <h3>Você está no <span id="current-room">Lobby</span></h3>

    <!-- Botões para mudar de sala (incluindo Lobby) -->
    <div id="rooms">
      <!-- Será preenchido dinamicamente com as salas disponíveis -->
      <button class="room-button" data-room="lobby">Lobby</button>
    </div>

    <!-- Área onde as mensagens serão exibidas -->
    <ul id="messages"></ul>

    <!-- Input para digitar mensagem -->
    <div id="input-container">
      <input type="text" id="input" placeholder="Digite sua mensagem" />
      <button id="send">Enviar</button>
    </div>
  </div>

  <script>
    document.addEventListener('DOMContentLoaded', () => {
      const socket = io();

      const loginDiv    = document.getElementById('login');
      const chatAreaDiv = document.getElementById('chat-area');
      const usernameInp = document.getElementById('username');
      const btnLogin    = document.getElementById('btn-login');

      const roomsDiv       = document.getElementById('rooms');
      const currentRoomLbl = document.getElementById('current-room');
      const messagesList   = document.getElementById('messages');
      const inputMsg       = document.getElementById('input');
      const btnSend        = document.getElementById('send');

      let username = '';
      let currentRoom = 'lobby'; // por padrão, após login o usuário fica no ‘lobby’

      // 1. Evento de login
      btnLogin.onclick = () => {
        const nome = usernameInp.value.trim();
        if (!nome) {
          alert('Por favor, coloque um nome de usuário.');
          return;
        }
        username = nome;
        // Esconde a tela de login, mostra a de chat
        loginDiv.style.display = 'none';
        chatAreaDiv.style.display = 'block';

        // Informa ao servidor que estamos entrando no lobby
        socket.emit('join_lobby', { username });
      };

      // 2. Recebe lista de salas disponíveis
      socket.on('rooms_list', (data) => {
        const rooms = data.rooms || [];
        // Limpa div de rooms (exceto o botão “Lobby” que já existe)
        roomsDiv.innerHTML = '';
        // Botão para lobby
        const btnLobby = document.createElement('button');
        btnLobby.textContent = 'Lobby';
        btnLobby.dataset.room = 'lobby';
        btnLobby.classList.add('room-button');
        roomsDiv.appendChild(btnLobby);

        // Cria botão para cada sala
        rooms.forEach(roomName => {
          const btn = document.createElement('button');
          btn.textContent = roomName;
          btn.dataset.room = roomName;
          btn.classList.add('room-button');
          roomsDiv.appendChild(btn);
        });
      });

      // 3. Trocar de sala ao clicar em botão
      roomsDiv.addEventListener('click', (e) => {
        if (e.target.tagName === 'BUTTON') {
          const chosenRoom = e.target.dataset.room;
          if (chosenRoom === currentRoom) return; // já está nessa sala

          // Avisa ao servidor qual sala será deixada
          if (currentRoom !== 'lobby') {
            socket.emit('leave_room', { username, room: currentRoom });
          } else {
            // se estava no lobby, opcionalmente notificar saída do lobby
            socket.emit('leave_room', { username, room: 'lobby' });
          }

          // Limpa lista de mensagens na tela
          messagesList.innerHTML = '';

          // Atualiza rótulo de sala atual no cliente
          currentRoom = chosenRoom;
          currentRoomLbl.textContent = (currentRoom === 'lobby') ? 'Lobby' : currentRoom;

          // Emite evento para ingressar na nova sala
          if (currentRoom === 'lobby') {
            socket.emit('join_lobby', { username });
          } else {
            socket.emit('join_room', { username, room: currentRoom });
          }
        }
      });

      // 4. Receber mensagens enviadas pela sala atual
      socket.on('message', (msg) => {
        const li = document.createElement('li');
        li.textContent = msg;
        messagesList.appendChild(li);
        // Autoscroll para a mensagem mais recente
        messagesList.scrollTop = messagesList.scrollHeight;
      });

      // 5. Receber erros do servidor (e.g. sala não existe)
      socket.on('error_message', (data) => {
        alert(data.msg || 'Erro desconhecido.');
      });

      // 6. Enviar mensagem para a sala atual
      btnSend.onclick = () => {
        const text = inputMsg.value.trim();
        if (!text) return;
        socket.emit('send_message', {
          username,
          room: currentRoom,
          message: text
        });
        inputMsg.value = '';
      };

      // Também enviar mensagem ao pressionar Enter
      inputMsg.addEventListener('keyup', (e) => {
        if (e.key === 'Enter') btnSend.click();
      });
    });
  </script>
</body>
</html>
````

## 6. Tratamento de Eventos e Reconexão

- Defina eventos customizados: `@socketio.on('my_event')`.
- Reconexão automática no cliente Socket.IO.
- Adicione timeouts e heartbeats configuráveis.

## 7. Testando a Aplicação

1. Inicie o servidor: `python app.py`.
2. Abra `http://localhost:5000/` em várias abas para simular múltiplos clientes.
3. Envie mensagens e veja atualização em tempo real.

---

Parabéns! Você agora domina WebSockets em Python e pode criar aplicações real-time.